#### 1. **Model Runner Step Sync And Async Testing**

Notebook tests `ModelRunnerStep` within a serving graph to enable real-time monitoring and drift detection with MLRun. 
The main focus is to test te preformance of sync and async usage.

In [2]:
# Import mlrun and create project instance
import mlrun
mlrun.set_env_from_file('../../cust_cs.env')

image = "mlrun/mlrun"
project_name = "model-runner-tests"
project = mlrun.get_or_create_project(project_name, context="./",user_project=True, allow_cross_project=True)

> 2025-11-02 11:58:29,214 [info] Created and saved project: {"context":"./","from_template":null,"name":"model-runner-tests-matanz","overwrite":false,"save":true}
> 2025-11-02 11:58:29,214 [info] Project created successfully: {"project_name":"model-runner-tests-matanz","stored_in_db":true}


In [3]:
# Import tools
import pandas as pd
from sklearn.svm import SVC
import pickle
from sklearn.datasets import load_iris
from mlrun.features import Feature

In [4]:
# Train simple SVM model on Iris dataset, save it and reformat the DS as list
iris = load_iris()
clf = SVC()           
clf.fit(iris.data, iris.target)
with open("SVM.pkl", "wb") as fh:
    pickle.dump(clf, fh)
iris_data = iris["data"].tolist()

# load the dataset again as a DF
iris = load_iris()
train_set = pd.DataFrame(
    iris["data"],
    columns=["sepal_length_cm", "sepal_width_cm", "petal_length_cm", "petal_width_cm"],
)

# Create a Model Artifact in the project using the trained model
model_name = "SVM"
model_artifact = project.log_model(
    model_name,
    model_file="SVM.pkl",
    training_set=train_set,
    framework="sklearn",
    outputs=[Feature(name="label")],
)

#### 5. Define your function and ModelRunnerStep

Define functions to all the edge cases

In [5]:
# Config the code path and the serving function sync and async
bouth_code_path = r"model_class_bouth.py"

function = project.set_function(func=bouth_code_path,image="mlrun/mlrun",kind="serving",name="bouth")

#set the function workers to 10 so it will be able to run 10 graphs
function.with_http(workers=10)

> 2025-11-02 11:58:34,006 [warning] Adding HTTP trigger despite the default HTTP trigger creation being disabled


In [6]:
from mlrun.serving.states import ModelRunnerStep

model_runner_step = ModelRunnerStep(
    name="my_runner", model_selector="MyModelSelector",model_selector_parameters={"name":"my-selector"})

for i in range(10):
    model_runner_step.add_model(
        model_class="MyModel",
        endpoint_name=f"my-{i}-model",
        model_artifact=model_artifact,
        input_path="inputs.here",
        result_path="outputs",
        outputs=["label"],
        execution_mechanism="thread_pool",
    )

#### 6. Build graphs to all the edge cases

description


In [7]:
async_graph_undefined = function.set_topology("flow",engine="async")
async_graph_undefined.to("MyPreprocessStep").to(model_runner_step).to("MyEnrichStep").respond()
async_graph_undefined.plot()


ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [8]:
from random import choice
from datetime import datetime

iris_data = iris["data"].tolist()
data_point = choice(iris_data)
print(f"Data point:{data_point}")

Data point:[5.1, 2.5, 3.0, 1.1]


# Invoke X times in perallel

#### 7. Deploying Your Function

Running this cell will deploy your serving function to the cluster. This also deploys the real-time monitoring functions for your project, which are configured to track the serving function's performance and detect model drift.

In [9]:
adr = function.deploy()

> 2025-11-02 11:58:34,494 [info] Starting remote function deploy
2025-11-02 09:58:35  (info) Deploying function
2025-11-02 09:58:35  (info) Building
2025-11-02 09:58:35  (info) Staging files and preparing base images
2025-11-02 09:58:35  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2025-11-02 09:58:35  (info) Building processor image
2025-11-02 10:00:00  (info) Build complete
2025-11-02 10:00:36  (info) Function deploy complete
> 2025-11-02 12:00:45,050 [info] Model endpoint creation task completed with state succeeded
> 2025-11-02 12:00:45,051 [info] Successfully deployed function: {"external_invocation_urls":["model-runner-tests-matanz-bouth.default-tenant.app.cust-cs-il.iguazio-cd0.com/"],"internal_invocation_urls":["nuclio-model-runner-tests-matanz-bouth.default-tenant.svc.cluster.local:8080"]}


In [10]:
import asyncio
from concurrent.futures import ThreadPoolExecutor

def call_test():
    return function.invoke(
        "/",
        body={
            "models": None,
            "inputs": [data_point, data_point],
        },
    )

async def run_tests():
    loop = asyncio.get_running_loop()
    with ThreadPoolExecutor() as executor:
        # Schedule 10 synchronous calls to run concurrently in threads
        tasks = [loop.run_in_executor(executor, call_test) for _ in range(10)]
        results = await asyncio.gather(*tasks)
    print("All results:", results)
    return results

# If you're inside a running loop (like Jupyter), use:
time_start = datetime.now()
results = await run_tests()
time_end = datetime.now()
Time_taken = time_end - time_start

All results: [{'my-0-model': {'outputs': {'label': [1, 1]}}, 'my-1-model': {'outputs': {'label': [1, 1]}}, 'my-2-model': {'outputs': {'label': [1, 1]}}, 'my-3-model': {'outputs': {'label': [1, 1]}}, 'my-4-model': {'outputs': {'label': [1, 1]}}, 'my-5-model': {'outputs': {'label': [1, 1]}}, 'my-6-model': {'outputs': {'label': [1, 1]}}, 'my-7-model': {'outputs': {'label': [1, 1]}}, 'my-8-model': {'outputs': {'label': [1, 1]}}, 'my-9-model': {'outputs': {'label': [1, 1]}}, 'timestamp': '1762077648.770317'}, {'my-0-model': {'outputs': {'label': [1, 1]}}, 'my-1-model': {'outputs': {'label': [1, 1]}}, 'my-2-model': {'outputs': {'label': [1, 1]}}, 'my-3-model': {'outputs': {'label': [1, 1]}}, 'my-4-model': {'outputs': {'label': [1, 1]}}, 'my-5-model': {'outputs': {'label': [1, 1]}}, 'my-6-model': {'outputs': {'label': [1, 1]}}, 'my-7-model': {'outputs': {'label': [1, 1]}}, 'my-8-model': {'outputs': {'label': [1, 1]}}, 'my-9-model': {'outputs': {'label': [1, 1]}}, 'timestamp': '1762077648.7178

In [11]:
print("Time taken:", Time_taken)

Time taken: 0:00:02.617129


In [12]:
results

[{'my-0-model': {'outputs': {'label': [1, 1]}},
  'my-1-model': {'outputs': {'label': [1, 1]}},
  'my-2-model': {'outputs': {'label': [1, 1]}},
  'my-3-model': {'outputs': {'label': [1, 1]}},
  'my-4-model': {'outputs': {'label': [1, 1]}},
  'my-5-model': {'outputs': {'label': [1, 1]}},
  'my-6-model': {'outputs': {'label': [1, 1]}},
  'my-7-model': {'outputs': {'label': [1, 1]}},
  'my-8-model': {'outputs': {'label': [1, 1]}},
  'my-9-model': {'outputs': {'label': [1, 1]}},
  'timestamp': '1762077648.770317'},
 {'my-0-model': {'outputs': {'label': [1, 1]}},
  'my-1-model': {'outputs': {'label': [1, 1]}},
  'my-2-model': {'outputs': {'label': [1, 1]}},
  'my-3-model': {'outputs': {'label': [1, 1]}},
  'my-4-model': {'outputs': {'label': [1, 1]}},
  'my-5-model': {'outputs': {'label': [1, 1]}},
  'my-6-model': {'outputs': {'label': [1, 1]}},
  'my-7-model': {'outputs': {'label': [1, 1]}},
  'my-8-model': {'outputs': {'label': [1, 1]}},
  'my-9-model': {'outputs': {'label': [1, 1]}},
  '